# Data Cleaning with R

This notebook demonstrates data cleaning principles in R. First we take a look at some encoding issues using a contrived example, and then we work through an example of loading some data as published on the Open Data portal.

- [Part 1: Encoding Issues](#Encoding-Issues)
- [Part 2: Example: Open Data](#Example:-Open-Data)

In [1]:
library(readr)
library(stringr)
library(dplyr)
library(tidyr)


Attaching package: 'dplyr'

The following objects are masked from 'package:stats':

    filter, lag

The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union



# Encoding Issues
Computers ultimately represent everything as a number, and over time, different operating systems, and different languages there have been many ways of representing text. Most encodings (except for ones designed specifically for Asian languages) are extensions of ASCII, which was designed for English, so the English alphabet and most common punctuation characters are usually not a problem, but ASCII does not include things like accented characters for French.

To demonstrate some of the potential issues, I made a file in Notepad and saved it using the default ANSI encoding (this default is changing to UTF8 as of the May 2019 update of Windows 10). This shows French text with accents, and non-ASCII punctuation characters. This formatting is similar to Excel's settings when used in French to generate a CSV file - it uses a ; instead of , for the delimiter as , is the decimal separator.

```
DEPARTEMENT;DÉPENSES;Punctuation
Administration du pipe-line du Nord;123456,88;“–”
Agence canadienne d’évaluation environnementale;999999,24;†•
Agence de développement économique du Canada pour les régions du Québec;234567,00;€
```

### Base R
Base R says it supports latin1 or UTF-8, with the note that control characters are interptreted as per Windows-1252 on some systems and particularly since R 3.5.0. In addition to `read.csv` and `read.delim` (for more control), R provides `read.csv2` to handle the European style CSVs with ; as the delimiter. This function does exactly what we want for this file by default.

In [2]:
read.csv2('data/NotepadFrench.csv')

Warning message in read.table(file = file, header = header, sep = sep, quote = quote, :
"incomplete final line found by readTableHeader on 'NotepadFrench.csv'"

DEPARTEMENT,DÉPENSES,Punctuation
Administration du pipe-line du Nord,123456.9,“–”
Agence canadienne d’évaluation environnementale,999999.2,†•
Agence de développement économique du Canada pour les régions du Québec,234567.0,€


## Readr package from Tidyverse
The Tidyverse is a popular set of R packages which include improvements to the Base R packages. Readr reads files into a 'tibble' rather than a data.frame, and it defaults to UTF-8. A cheat sheet for readr and tibble can be found here: https://rawgit.com/rstudio/cheatsheets/master/data-import.pdf

## Issue 1: Can't read the file at all with invalid UTF-8

In many cases, just calling read_csv will work, but not in this one.

In [3]:
read_csv('data/NotepadFrench.csv')

Parsed with column specification:
cols(
  `DEPARTEMENT;D<c9>PENSES;Punctuation` = col_character()
)
Warning message in rbind(names(probs), probs_f):
"number of columns of result is not a multiple of vector length (arg 1)"Warning message:
"3 parsing failures.
row # A tibble: 3 x 5 col     row col   expected  actual    file                expected   <int> <chr> <chr>     <chr>     <chr>               actual 1     1 <NA>  1 columns 2 columns 'NotepadFrench.csv' file 2     2 <NA>  1 columns 2 columns 'NotepadFrench.csv' row 3     3 <NA>  1 columns 2 columns 'NotepadFrench.csv'
"Warning message in FUN(X[[i]], ...):
"input string 1 is invalid UTF-8"Warning message in FUN(X[[i]], ...):
"input string 1 is invalid UTF-8"Warning message in FUN(X[[i]], ...):
"input string 1 is invalid UTF-8"Warning message in FUN(X[[i]], ...):
"input string 1 is invalid UTF-8"Warning message in FUN(X[[i]], ...):
"input string 1 is invalid UTF-8"Warning message in FUN(X[[i]], ...):
"input string 1 is invalid UTF-8

In order to set the encoding for Readr, we need to create a [locale](https://readr.tidyverse.org/reference/locale.html). We also switch to using `read_csv2` which deals with the European style CSV with ; as the separator and , as the decimal. `read_delim` is another option providing more control.

## Issue 2: Not all characters loaded properly - character codes and mojibake
`latin-1` (also known as `ISO-8859-1`) is a common encoding for Western European text including French, but it doesn't include the punctuation characters. Here R is showing the punctuation haracters as strings like <U+0093>. Note that Readr doesn't use the same flexible definition of latin1 as the built in read.csv.

In [4]:
no_punctuation = read_csv2('data/NotepadFrench.csv', locale=locale(encoding="latin1"))
no_punctuation

Using ',' as decimal and '.' as grouping mark. Use read_delim() for more control.
Parsed with column specification:
cols(
  DEPARTEMENT = col_character(),
  DÉPENSES = col_double(),
  Punctuation = col_character()
)


DEPARTEMENT,DÉPENSES,Punctuation
Administration du pipe-line du Nord,123456.9,<U+0093><U+0096><U+0094>
Agence canadienne d<U+0092>évaluation environnementale,999999.2,<U+0086><U+0095>
Agence de développement économique du Canada pour les régions du Québec,234567.0,<U+0080>


Another possibility is that characters map to incorrect visible characters. Here we attempt to load it using IBM863, 'MS-DOS French Canada', which was used by MS-DOS systems in Canada. This also demonstrates using `read_delim` for the same results as `read_csv2`.

In [5]:
read_delim('data/NotepadFrench.csv', ";", locale=locale(encoding="IBM863", decimal=","))

Parsed with column specification:
cols(
  DEPARTEMENT = col_character(),
  `D+PENSES` = col_double(),
  Punctuation = col_character()
)


DEPARTEMENT,D╔PENSES,Punctuation
Administration du pipe-line du Nord,123456.9,ôûË
Agence canadienne dÊTvaluation environnementale,999999.2,¶Ï
Agence de dTveloppement Tconomique du Canada pour les rTgions du QuTbec,234567.0,Ç


`windows-1252` is a Windows specific superset of `ISO-8859-1` which includes several additional characters. It is one of several different 'ANSI Codepages' for representing different languages. This is the most likely option if a file was created on a Windows system in English, French, or most Western European languages. Here I'm using `read_delim` without setting the decimal option to demonstrate what happens in R.

In [6]:
wrong_decimal = read_delim('data/NotepadFrench.csv', ";", locale=locale(encoding="windows-1252"))
wrong_decimal

Parsed with column specification:
cols(
  DEPARTEMENT = col_character(),
  DÉPENSES = col_number(),
  Punctuation = col_character()
)


DEPARTEMENT,DÉPENSES,Punctuation
Administration du pipe-line du Nord,12345688,“–”
Agence canadienne d’évaluation environnementale,99999924,†•
Agence de développement économique du Canada pour les régions du Québec,23456700,€


While not strictly an encoding issue, one other issue with dealing with multiple languages is different localization settings like date formats and decimal separators. Here, ',' is being treated as a thousands separator not a decimal - unlike in Python it did read it as a number, but not the right number! This can be set by using read_csv2, or passing in parameters to the locale.

In [7]:
sum(wrong_decimal["DÉPENSES"])

[1] 135802312

In [8]:
correct_sample = read_csv2('data/NotepadFrench.csv', locale=locale(encoding="windows-1252"))
correct_sample

Using ',' as decimal and '.' as grouping mark. Use read_delim() for more control.
Parsed with column specification:
cols(
  DEPARTEMENT = col_character(),
  DÉPENSES = col_double(),
  Punctuation = col_character()
)


DEPARTEMENT,DÉPENSES,Punctuation
Administration du pipe-line du Nord,123456.9,“–”
Agence canadienne d’évaluation environnementale,999999.2,†•
Agence de développement économique du Canada pour les régions du Québec,234567.0,€


In [9]:
sum(correct_sample["DÉPENSES"])

[1] 1358023

Here we save a version of the file which plays nicely with Readr defaults.

In [10]:
write.csv(correct_sample, 'data/utf8FrenchR.csv', fileEncoding='utf8', row.names=FALSE)
read_csv('data/utf8FrenchR.csv')

Parsed with column specification:
cols(
  DEPARTEMENT = col_character(),
  DÉPENSES = col_double(),
  Punctuation = col_character()
)


DEPARTEMENT,DÉPENSES,Punctuation
Administration du pipe-line du Nord,123456.9,“–”
Agence canadienne d’évaluation environnementale,999999.2,†•
Agence de développement économique du Canada pour les régions du Québec,234567.0,€


## Issue 3: Loading UTF8 into Excel

Excel defaults to the ANSI codepage used by the operating system (e.g. Windows-1252), and will not properly load UTF8 text via File->Open.

The solution is to import the text into Excel instead and choose the appropriate encoding.
![Loading UTF8 into Excel using Text Import](ExcelImport.png)

# Example: Open Data
Fuel consumption ratings (NRCAN): https://open.canada.ca/data/en/dataset/98f1a129-f628-4ce4-b24d-6f16bf24dd64

In this scenario, I want to do some analysis on a dataset published to the open data portal. This dataset is spread over multiple files for different years, and there are some minor changes to the columns over time. Like many datasets on Open Data it has some features which make it easier for people to look at but harder for machines - the column names are formatted over two rows, and there is some extra descriptions of fields below the main data section. A bit more unusually, it also has extra blank lines between rows and for some years, there are lots of extra blank columns and rows at the end.

The Python version of this notebook has a functions which downloads the files, here we just read in.

In [11]:
en_consumption_files = list.files(pattern="downloads/MY[0-9]{4} Fuel Consumption Ratings.*csv")
en_consumption_files

character(0)

## Cleaning up just the most recent year step by step
In this section, I go through the steps to look at and fix issues with one file.

In [12]:
data = read_csv("downloads/MY2019 Fuel Consumption Ratings.csv")
head(data)

Warning message:
"Missing column names filled in: 'X10' [10], 'X11' [11], 'X12' [12], 'X16' [16], 'X17' [17], 'X18' [18], 'X19' [19], 'X20' [20], 'X21' [21], 'X22' [22], 'X23' [23], 'X24' [24], 'X25' [25], 'X26' [26], 'X27' [27], 'X28' [28], 'X29' [29], 'X30' [30], 'X31' [31], 'X32' [32], 'X33' [33], 'X34' [34], 'X35' [35], 'X36' [36], 'X37' [37], 'X38' [38], 'X39' [39], 'X40' [40], 'X41' [41], 'X42' [42], 'X43' [43], 'X44' [44], 'X45' [45], 'X46' [46], 'X47' [47], 'X48' [48], 'X49' [49], 'X50' [50], 'X51' [51], 'X52' [52], 'X53' [53], 'X54' [54], 'X55' [55], 'X56' [56], 'X57' [57], 'X58' [58], 'X59' [59], 'X60' [60], 'X61' [61], 'X62' [62], 'X63' [63], 'X64' [64], 'X65' [65], 'X66' [66], 'X67' [67], 'X68' [68], 'X69' [69], 'X70' [70], 'X71' [71], 'X72' [72], 'X73' [73], 'X74' [74], 'X75' [75], 'X76' [76], 'X77' [77], 'X78' [78], 'X79' [79], 'X80' [80], 'X81' [81], 'X82' [82], 'X83' [83], 'X84' [84], 'X85' [85], 'X86' [86], 'X87' [87], 'X88' [88], 'X89' [89], 'X90' [90], 'X91' [91], 'X

Model,Make,Model_1,Vehicle Class,Engine Size,Cylinders,Transmission,Fuel,Fuel Consumption,X10,...,X212,X213,X214,X215,X216,X217,X218,X219,X220,X221
Year,NA,NA,NA,(L),NA,NA,Type,City (L/100 km),Hwy (L/100 km),...,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
2019,Acura,ILX,Compact,2.4,4,AM8,Z,9.9,7.0,...,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
2019,Acura,MDX SH-AWD,SUV: Small,3.5,6,AS9,Z,12.2,9.0,...,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
2019,Acura,MDX SH-AWD A-SPEC,SUV: Small,3.5,6,AS9,Z,12.2,9.5,...,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
2019,Acura,MDX Hybrid AWD,SUV: Small,3.0,6,AM7,Z,9.1,9.0,...,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
2019,Acura,NSX,Two-seater,3.5,6,AM9,Z,11.1,10.8,...,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA


In [13]:
tail(data)

Model,Make,Model_1,Vehicle Class,Engine Size,Cylinders,Transmission,Fuel,Fuel Consumption,X10,...,X212,X213,X214,X215,X216,X217,X218,X219,X220,X221
NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,...,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,...,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,...,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,...,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,...,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,...,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA


Looking at the top and the bottom of the file, I see that there are a lot of columns that look to be empty, and also a lot of empty rows at the bottom.

### Identify and drop the extra columns
Select all columns that have any values that aren't NA

In [14]:
data = data %>% select_if(function(x) any(!is.na(x)))
head(data)

Model,Make,Model_1,Vehicle Class,Engine Size,Cylinders,Transmission,Fuel,Fuel Consumption,X10,X11,X12,CO2 Emissions,CO2,Smog
Year,NA,NA,NA,(L),NA,NA,Type,City (L/100 km),Hwy (L/100 km),Comb (L/100 km),Comb (mpg),(g/km),Rating,Rating
2019,Acura,ILX,Compact,2.4,4,AM8,Z,9.9,7.0,8.6,33,199,6,3
2019,Acura,MDX SH-AWD,SUV: Small,3.5,6,AS9,Z,12.2,9.0,10.8,26,252,4,3
2019,Acura,MDX SH-AWD A-SPEC,SUV: Small,3.5,6,AS9,Z,12.2,9.5,11.0,26,258,4,3
2019,Acura,MDX Hybrid AWD,SUV: Small,3.0,6,AM7,Z,9.1,9.0,9.0,31,210,5,3
2019,Acura,NSX,Two-seater,3.5,6,AM9,Z,11.1,10.8,11.0,26,261,4,3


In [15]:
str(data)

Classes 'tbl_df', 'tbl' and 'data.frame':	1283 obs. of  15 variables:
 $ Model           : chr  "Year" "2019" "2019" "2019" ...
 $ Make            : chr  NA "Acura" "Acura" "Acura" ...
 $ Model_1         : chr  NA "ILX" "MDX SH-AWD" "MDX SH-AWD A-SPEC" ...
 $ Vehicle Class   : chr  NA "Compact" "SUV: Small" "SUV: Small" ...
 $ Engine Size     : chr  "(L)" "2.4" "3.5" "3.5" ...
 $ Cylinders       : int  NA 4 6 6 6 6 4 4 6 4 ...
 $ Transmission    : chr  NA "AM8" "AS9" "AS9" ...
 $ Fuel            : chr  "Type" "Z" "Z" "Z" ...
 $ Fuel Consumption: chr  "City (L/100 km)" "9.9" "12.2" "12.2" ...
 $ X10             : chr  "Hwy (L/100 km)" "7.0" "9.0" "9.5" ...
 $ X11             : chr  "Comb (L/100 km)" "8.6" "10.8" "11.0" ...
 $ X12             : chr  "Comb (mpg)" "33" "26" "26" ...
 $ CO2 Emissions   : chr  "(g/km)" "199" "252" "258" ...
 $ CO2             : chr  "Rating" "6" "4" "4" ...
 $ Smog            : chr  "Rating" "3" "3" "3" ...
 - attr(*, "spec")=List of 2
  ..$ cols   :List of 

### Update the column names and remove the extra row
This code looks the column names (which come from the first row of the file) and the first data row. Some columns have useful information spread over both rows, and some in only the first or second row.

In [16]:
names = colnames(data)
names

[1] "Model"            "Make"             "Model_1"          "Vehicle Class"   
 [5] "Engine Size"      "Cylinders"        "Transmission"     "Fuel"            
 [9] "Fuel Consumption" "X10"              "X11"              "X12"             
[13] "CO2 Emissions"    "CO2"              "Smog"

In [17]:
# Update column names
clean_column_names <- function(data) {
    names = colnames(data)
    new_columns = rep("", length(names))

    for(i in 1:length(names)) {
        new_name = ""
        first_row_val = data[1, i]
        col = names[i]
        if (startsWith(col, "X")) {
            new_name = first_row_val
            }
        else if(is.na(first_row_val)) {
            new_name = col
            }
        else {
            new_name = str_c(col, first_row_val, sep=" ")
            }
        new_columns[i] = new_name
    }
    new_columns
}

new_columns <- clean_column_names(data)
new_columns

[[1]]
[1] "Model Year"

[[2]]
[1] "Make"

[[3]]
[1] "Model_1"

[[4]]
[1] "Vehicle Class"

[[5]]
[1] "Engine Size (L)"

[[6]]
[1] "Cylinders"

[[7]]
[1] "Transmission"

[[8]]
[1] "Fuel Type"

[[9]]
[1] "Fuel Consumption City (L/100 km)"

[[10]]
[1] "Hwy (L/100 km)"

[[11]]
[1] "Comb (L/100 km)"

[[12]]
[1] "Comb (mpg)"

[[13]]
[1] "CO2 Emissions (g/km)"

[[14]]
[1] "CO2 Rating"

[[15]]
[1] "Smog Rating"

In [18]:
map_column <- function(original_name) {
    i <- which(original_name == names)
    return(new_columns[i])
}

map_column("X11")

[[1]]
[1] "Comb (L/100 km)"

In [19]:
data <- data %>% rename_all(map_column)

### Drop rows where anything is missing

In [20]:
data = drop_na(data)
data

Model Year,Make,Model_1,Vehicle Class,Engine Size (L),Cylinders,Transmission,Fuel Type,Fuel Consumption City (L/100 km),Hwy (L/100 km),Comb (L/100 km),Comb (mpg),CO2 Emissions (g/km),CO2 Rating,Smog Rating
2019,Acura,ILX,Compact,2.4,4,AM8,Z,9.9,7.0,8.6,33,199,6,3
2019,Acura,MDX SH-AWD,SUV: Small,3.5,6,AS9,Z,12.2,9.0,10.8,26,252,4,3
2019,Acura,MDX SH-AWD A-SPEC,SUV: Small,3.5,6,AS9,Z,12.2,9.5,11.0,26,258,4,3
2019,Acura,MDX Hybrid AWD,SUV: Small,3.0,6,AM7,Z,9.1,9.0,9.0,31,210,5,3
2019,Acura,NSX,Two-seater,3.5,6,AM9,Z,11.1,10.8,11.0,26,261,4,3
2019,Acura,RDX AWD,SUV: Small,2.0,4,AS10,Z,11.0,8.6,9.9,29,232,5,6
2019,Acura,RDX AWD A-SPEC,SUV: Small,2.0,4,AS10,Z,11.3,9.1,10.3,27,242,5,6
2019,Acura,RLX Hybrid,Mid-size,3.5,6,AM7,Z,8.4,8.2,8.4,34,196,6,7
2019,Acura,TLX,Compact,2.4,4,AM8,Z,10.0,7.1,8.7,32,205,6,3
2019,Acura,TLX A-SPEC,Compact,2.4,4,AM8,Z,10.2,7.4,8.9,32,209,5,3


In [21]:
summary(data)

  Model Year            Make             Model_1          Vehicle Class     
 Length:1022        Length:1022        Length:1022        Length:1022       
 Class :character   Class :character   Class :character   Class :character  
 Mode  :character   Mode  :character   Mode  :character   Mode  :character  
                                                                            
                                                                            
                                                                            
 Engine Size (L)      Cylinders      Transmission        Fuel Type        
 Length:1022        Min.   : 3.000   Length:1022        Length:1022       
 Class :character   1st Qu.: 4.000   Class :character   Class :character  
 Mode  :character   Median : 6.000   Mode  :character   Mode  :character  
                    Mean   : 5.588                                        
                    3rd Qu.: 6.000                                        
           

There are other columns that look numeric, why aren't they here?

Many columns got imported as 'character', meaning they are interptreted as text. Most of this is because of that 2 row header.

### Convert to numeric:
This converts the listed columns to numeric values.

In [22]:
numeric_cols = c("Model Year", "Engine Size (L)", "Fuel Consumption City (L/100 km)", "Hwy (L/100 km)", "Comb (L/100 km)", "Comb (mpg)", "CO2 Emissions (g/km)", "CO2 Rating", "Smog Rating")
str(data %>% mutate_at(numeric_cols,as.numeric))


Classes 'tbl_df', 'tbl' and 'data.frame':	1022 obs. of  15 variables:
 $ Model Year                      : num  2019 2019 2019 2019 2019 ...
 $ Make                            : chr  "Acura" "Acura" "Acura" "Acura" ...
 $ Model_1                         : chr  "ILX" "MDX SH-AWD" "MDX SH-AWD A-SPEC" "MDX Hybrid AWD" ...
 $ Vehicle Class                   : chr  "Compact" "SUV: Small" "SUV: Small" "SUV: Small" ...
 $ Engine Size (L)                 : num  2.4 3.5 3.5 3 3.5 2 2 3.5 2.4 2.4 ...
 $ Cylinders                       : int  4 6 6 6 6 4 4 6 4 4 ...
 $ Transmission                    : chr  "AM8" "AS9" "AS9" "AM7" ...
 $ Fuel Type                       : chr  "Z" "Z" "Z" "Z" ...
 $ Fuel Consumption City (L/100 km): num  9.9 12.2 12.2 9.1 11.1 11 11.3 8.4 10 10.2 ...
 $ Hwy (L/100 km)                  : num  7 9 9.5 9 10.8 8.6 9.1 8.2 7.1 7.4 ...
 $ Comb (L/100 km)                 : num  8.6 10.8 11 9 11 9.9 10.3 8.4 8.7 8.9 ...
 $ Comb (mpg)                      : num  33 26 26 3

## Putting it all together to look at more years

In [23]:
a = c("a", "b")
b = c("b", "c")
intersect(a, b)

[1] "b"

In [24]:
clean = function(df) {
    df <- df %>% select_if(function(x) any(!is.na(x))) %>% drop_na
    names = colnames(df)
    new_names = c('Year', 'Make', 'Model', 'Class', 'Engine Size',
        'Cylinders', 'Transmission', 'Fuel Type',
        'Fuel Consumption City', 'Fuel Consumption Hwy', 'Fuel Consumption Comb',
        'Comb (mpg)', 'CO2 Emissions', 'CO2 Rating', 'Smog Rating')
    map_names <- function(original_name) {
        i <- which(original_name == names)
        return(new_names[i])
    }
   

    df <- df %>% rename_all(map_names)  
                           
    columns_to_convert = c("Year", "Engine Size", "Fuel Consumption City",'Fuel Consumption Hwy', 'Fuel Consumption Comb', "Comb (mpg)", "CO2 Emissions", "CO2 Rating", "Smog Rating")
    numeric_cols <- intersect(columns_to_convert, colnames(df)) # The columns from numeric_cols that exist in this dataset
    df <- df %>% mutate_at(numeric_cols,as.numeric)
    df 
}

clean(read_csv("downloads/MY2019 Fuel Consumption Ratings.csv"))

Warning message:
"Missing column names filled in: 'X10' [10], 'X11' [11], 'X12' [12], 'X16' [16], 'X17' [17], 'X18' [18], 'X19' [19], 'X20' [20], 'X21' [21], 'X22' [22], 'X23' [23], 'X24' [24], 'X25' [25], 'X26' [26], 'X27' [27], 'X28' [28], 'X29' [29], 'X30' [30], 'X31' [31], 'X32' [32], 'X33' [33], 'X34' [34], 'X35' [35], 'X36' [36], 'X37' [37], 'X38' [38], 'X39' [39], 'X40' [40], 'X41' [41], 'X42' [42], 'X43' [43], 'X44' [44], 'X45' [45], 'X46' [46], 'X47' [47], 'X48' [48], 'X49' [49], 'X50' [50], 'X51' [51], 'X52' [52], 'X53' [53], 'X54' [54], 'X55' [55], 'X56' [56], 'X57' [57], 'X58' [58], 'X59' [59], 'X60' [60], 'X61' [61], 'X62' [62], 'X63' [63], 'X64' [64], 'X65' [65], 'X66' [66], 'X67' [67], 'X68' [68], 'X69' [69], 'X70' [70], 'X71' [71], 'X72' [72], 'X73' [73], 'X74' [74], 'X75' [75], 'X76' [76], 'X77' [77], 'X78' [78], 'X79' [79], 'X80' [80], 'X81' [81], 'X82' [82], 'X83' [83], 'X84' [84], 'X85' [85], 'X86' [86], 'X87' [87], 'X88' [88], 'X89' [89], 'X90' [90], 'X91' [91], 'X

Year,Make,Model,Class,Engine Size,Cylinders,Transmission,Fuel Type,Fuel Consumption City,Fuel Consumption Hwy,Fuel Consumption Comb,Comb (mpg),CO2 Emissions,CO2 Rating,Smog Rating
2019,Acura,ILX,Compact,2.4,4,AM8,Z,9.9,7.0,8.6,33,199,6,3
2019,Acura,MDX SH-AWD,SUV: Small,3.5,6,AS9,Z,12.2,9.0,10.8,26,252,4,3
2019,Acura,MDX SH-AWD A-SPEC,SUV: Small,3.5,6,AS9,Z,12.2,9.5,11.0,26,258,4,3
2019,Acura,MDX Hybrid AWD,SUV: Small,3.0,6,AM7,Z,9.1,9.0,9.0,31,210,5,3
2019,Acura,NSX,Two-seater,3.5,6,AM9,Z,11.1,10.8,11.0,26,261,4,3
2019,Acura,RDX AWD,SUV: Small,2.0,4,AS10,Z,11.0,8.6,9.9,29,232,5,6
2019,Acura,RDX AWD A-SPEC,SUV: Small,2.0,4,AS10,Z,11.3,9.1,10.3,27,242,5,6
2019,Acura,RLX Hybrid,Mid-size,3.5,6,AM7,Z,8.4,8.2,8.4,34,196,6,7
2019,Acura,TLX,Compact,2.4,4,AM8,Z,10.0,7.1,8.7,32,205,6,3
2019,Acura,TLX A-SPEC,Compact,2.4,4,AM8,Z,10.2,7.4,8.9,32,209,5,3


In [25]:
my2000 = read_csv("downloads/MY2000 Fuel Consumption Ratings 5-cycle.csv")


Warning message:
"Missing column names filled in: 'X10' [10], 'X11' [11], 'X12' [12]"Warning message:
"Duplicated column names deduplicated: 'MODEL' => 'MODEL_1' [3]"Parsed with column specification:
cols(
  MODEL = col_character(),
  MAKE = col_character(),
  MODEL_1 = col_character(),
  `VEHICLE CLASS` = col_character(),
  `ENGINE SIZE` = col_character(),
  CYLINDERS = col_integer(),
  TRANSMISSION = col_character(),
  FUEL = col_character(),
  `FUEL CONSUMPTION*` = col_character(),
  X10 = col_character(),
  X11 = col_character(),
  X12 = col_character(),
  `CO2 EMISSIONS` = col_character()
)


In [26]:
head(my2000)

MODEL,MAKE,MODEL_1,VEHICLE CLASS,ENGINE SIZE,CYLINDERS,TRANSMISSION,FUEL,FUEL CONSUMPTION*,X10,X11,X12,CO2 EMISSIONS
YEAR,NA,# = high output engine,NA,(L),NA,NA,TYPE,CITY (L/100 km),HWY (L/100 km),COMB (L/100 km),COMB (mpg),(g/km)
2000,ACURA,1.6EL,COMPACT,1.6,4,A4,X,10.5,8,9.4,30,216
2000,ACURA,1.6EL,COMPACT,1.6,4,M5,X,9.8,7.8,8.9,32,205
2000,ACURA,3.2TL,MID-SIZE,3.2,6,AS5,Z,13.7,8.8,11.5,25,265
2000,ACURA,3.5RL,MID-SIZE,3.5,6,A4,Z,15,10.9,13.1,22,301
2000,ACURA,INTEGRA,SUBCOMPACT,1.8,4,A4,X,11.4,8.3,10,28,230


In [27]:
clean(my2000)

Year,Make,Model,Class,Engine Size,Cylinders,Transmission,Fuel Type,Fuel Consumption City,Fuel Consumption Hwy,Fuel Consumption Comb,Comb (mpg),CO2 Emissions
2000,ACURA,1.6EL,COMPACT,1.6,4,A4,X,10.5,8.0,9.4,30,216
2000,ACURA,1.6EL,COMPACT,1.6,4,M5,X,9.8,7.8,8.9,32,205
2000,ACURA,3.2TL,MID-SIZE,3.2,6,AS5,Z,13.7,8.8,11.5,25,265
2000,ACURA,3.5RL,MID-SIZE,3.5,6,A4,Z,15.0,10.9,13.1,22,301
2000,ACURA,INTEGRA,SUBCOMPACT,1.8,4,A4,X,11.4,8.3,10.0,28,230
2000,ACURA,INTEGRA,SUBCOMPACT,1.8,4,M5,X,10.6,8.1,9.5,30,219
2000,ACURA,INTEGRA GSR/TYPE R,SUBCOMPACT,1.8,4,M5,Z,10.8,8.3,9.7,29,223
2000,ACURA,NSX,SUBCOMPACT,3.0,6,AS4,Z,15.2,10.9,13.3,21,306
2000,ACURA,NSX,SUBCOMPACT,3.2,6,M6,Z,15.4,10.7,13.3,21,306
2000,AUDI,A4,COMPACT,1.8,4,A5,Z,12.9,8.6,10.9,26,251


In [28]:
load_all_years <- function(files) {
    all_data_list <- lapply(files, function(f) { suppressWarnings(clean(read_csv(f)))})
    #all_data_list
    bind_rows(all_data_list)
}

In [29]:
combined <- load_all_years(en_consumption_files)

In [30]:
summary(combined)

< table of extent 0 x 0 >

And now I want to do all that again but using the French data files - maybe to compare that they're the same. Now that it's all set up, it's easy!

In [31]:
fr_consumption_files <- list.files(pattern="downloads/MY[0-9]{4} Fuel Consumption Ratings.*csv")
fr_combined <- load_all_years(fr_consumption_files)


In [32]:
summary(fr_combined)

< table of extent 0 x 0 >

## There's often more than one way to do it
Another way to deal with a two line header, or other scenarios where there is extra data at the start of a file, is to set the `col_names`, and `skip` options when reading in the file.

In [33]:
# It will deal with too many columns just fine
newer_colnames = c('Year', 'Make', 'Model', 'Class', 'Engine Size',
        'Cylinders', 'Transmission', 'Fuel Type',
        'Fuel Consumption City', 'Fuel Consumption Hwy', 'Fuel Consumption Comb',
        'Comb (mpg)', 'CO2 Emissions', 'CO2 Rating', 'Smog Rating')
read_csv("downloads/MY2019 Fuel Consumption Ratings.csv", col_names=newer_colnames, skip=3)

Parsed with column specification:
cols(
  Year = col_integer(),
  Make = col_character(),
  Model = col_character(),
  Class = col_character(),
  `Engine Size` = col_double(),
  Cylinders = col_integer(),
  Transmission = col_character(),
  `Fuel Type` = col_character(),
  `Fuel Consumption City` = col_double(),
  `Fuel Consumption Hwy` = col_double(),
  `Fuel Consumption Comb` = col_double(),
  `Comb (mpg)` = col_integer(),
  `CO2 Emissions` = col_integer(),
  `CO2 Rating` = col_integer(),
  `Smog Rating` = col_integer()
)
Warning message in rbind(names(probs), probs_f):
"number of columns of result is not a multiple of vector length (arg 1)"Warning message:
"1290 parsing failures.
row # A tibble: 5 x 5 col     row col   expected   actual      file                                       expected   <int> <chr> <chr>      <chr>       <chr>                                      actual 1     1 <NA>  15 columns 221 columns 'downloads/MY2019 Fuel Consumption Rating~ file 2     2 <NA>  15 colu

Year,Make,Model,Class,Engine Size,Cylinders,Transmission,Fuel Type,Fuel Consumption City,Fuel Consumption Hwy,Fuel Consumption Comb,Comb (mpg),CO2 Emissions,CO2 Rating,Smog Rating
2019,Acura,ILX,Compact,2.4,4,AM8,Z,9.9,7.0,8.6,33,199,6,3
2019,Acura,MDX SH-AWD,SUV: Small,3.5,6,AS9,Z,12.2,9.0,10.8,26,252,4,3
2019,Acura,MDX SH-AWD A-SPEC,SUV: Small,3.5,6,AS9,Z,12.2,9.5,11.0,26,258,4,3
2019,Acura,MDX Hybrid AWD,SUV: Small,3.0,6,AM7,Z,9.1,9.0,9.0,31,210,5,3
2019,Acura,NSX,Two-seater,3.5,6,AM9,Z,11.1,10.8,11.0,26,261,4,3
2019,Acura,RDX AWD,SUV: Small,2.0,4,AS10,Z,11.0,8.6,9.9,29,232,5,6
2019,Acura,RDX AWD A-SPEC,SUV: Small,2.0,4,AS10,Z,11.3,9.1,10.3,27,242,5,6
2019,Acura,RLX Hybrid,Mid-size,3.5,6,AM7,Z,8.4,8.2,8.4,34,196,6,7
2019,Acura,TLX,Compact,2.4,4,AM8,Z,10.0,7.1,8.7,32,205,6,3
2019,Acura,TLX A-SPEC,Compact,2.4,4,AM8,Z,10.2,7.4,8.9,32,209,5,3


In [34]:
read_csv("downloads/MY2000 Fuel Consumption Ratings 5-cycle.csv", col_names=newer_colnames, skip=3)

Parsed with column specification:
cols(
  Year = col_character(),
  Make = col_character(),
  Model = col_character(),
  Class = col_character(),
  `Engine Size` = col_double(),
  Cylinders = col_integer(),
  Transmission = col_character(),
  `Fuel Type` = col_character(),
  `Fuel Consumption City` = col_double(),
  `Fuel Consumption Hwy` = col_double(),
  `Fuel Consumption Comb` = col_double(),
  `Comb (mpg)` = col_integer(),
  `CO2 Emissions` = col_integer(),
  `CO2 Rating` = col_character(),
  `Smog Rating` = col_character()
)
Warning message in rbind(names(probs), probs_f):
"number of columns of result is not a multiple of vector length (arg 1)"Warning message:
"664 parsing failures.
row # A tibble: 5 x 5 col     row col   expected   actual     file                                        expected   <int> <chr> <chr>      <chr>      <chr>                                       actual 1     1 <NA>  15 columns 13 columns 'downloads/MY2000 Fuel Consumption Ratings~ file 2     2 <NA>  15

Year,Make,Model,Class,Engine Size,Cylinders,Transmission,Fuel Type,Fuel Consumption City,Fuel Consumption Hwy,Fuel Consumption Comb,Comb (mpg),CO2 Emissions,CO2 Rating,Smog Rating
2000,ACURA,1.6EL,COMPACT,1.6,4,A4,X,10.5,8.0,9.4,30,216,NA,NA
2000,ACURA,1.6EL,COMPACT,1.6,4,M5,X,9.8,7.8,8.9,32,205,NA,NA
2000,ACURA,3.2TL,MID-SIZE,3.2,6,AS5,Z,13.7,8.8,11.5,25,265,NA,NA
2000,ACURA,3.5RL,MID-SIZE,3.5,6,A4,Z,15.0,10.9,13.1,22,301,NA,NA
2000,ACURA,INTEGRA,SUBCOMPACT,1.8,4,A4,X,11.4,8.3,10.0,28,230,NA,NA
2000,ACURA,INTEGRA,SUBCOMPACT,1.8,4,M5,X,10.6,8.1,9.5,30,219,NA,NA
2000,ACURA,INTEGRA GSR/TYPE R,SUBCOMPACT,1.8,4,M5,Z,10.8,8.3,9.7,29,223,NA,NA
2000,ACURA,NSX,SUBCOMPACT,3.0,6,AS4,Z,15.2,10.9,13.3,21,306,NA,NA
2000,ACURA,NSX,SUBCOMPACT,3.2,6,M6,Z,15.4,10.7,13.3,21,306,NA,NA
2000,AUDI,A4,COMPACT,1.8,4,A5,Z,12.9,8.6,10.9,26,251,NA,NA
